Redis不使用表，它的数据库也不会预定义或者强制要求用户对`Redis`存储的不同数据进行关联

在使用类似`Redis`这样的内存数据库时，一个首先要考虑的问题就是“当服务器被关闭时，服务器的存储的数据将何去何从呢？`Redis`拥有两种不同形式的持久化方法，他们都可以用小而紧凑的格式将存储在内存中的数据写入硬盘：第一种持久化方法为时间点转储（`point-in-time dump`)，转储操作既可以在“指定时间段内有指定数量的写操作执行“这一条件被满足时执行，又可以通过调用两条转储到硬盘（`dump-to-disk`）命令中的任何一条来执行；第二种持久化方法将所有修改了数据库的命令都写入一个只追加（`append-only`）文件里面，用户可以根据数据的重要程度，将只追加写入设置为从不同步（`sync`）、每秒同步一次或者每写入一个命令就同步一次。

为了扩展`Redis`的读性能，并为`Redis`提供故障转移（`failover`）支持，`Redis`实现了主从复制特性：执行复制的从服务器回连接上主服务器，接受主服务器发送的整个数据库的初始副本；之后主服务器执行的写命令，都会被发送给所有连接的从服务器去执行，从而实时地更行从服务器的数据集，从而实时地更新从服务器的数据集。因为从服务器包含的数据回不断地进行更新，所以客户端可以向任意一个从服务器发送读请求，以此来避免对主服务器进行集中式的访问。

REDIS可以存储物种数据结构，分别为STRING, LIST, SET, ZSET

STRING: 可能是字符串、整数或者是浮点数。对整数和浮点数执行自增（increment）或者自减（decrement）操作

LIST： 一个列表，链表上每个节点都包含了一个字符串

SET：包含字符串的unordered collection，并且被包含的每个字符串都是独一无二，各不相同的

HASH：包含键值对的无序散表

ZSET：字符串成员（member）与浮点数分值（score）之间的有序映射，元素的排列书序由分值的大小决定

```redis
127.0.0.1:6379> set hello world
OK
127.0.0.1:6379> get hello
"world"
127.0.0.1:6379> del hello
(integer) 1
127.0.0.1:6379> get hello
(nil)
```

`LPUSH`命令和`RPUSH`命令分别用于将元素推入列表的左端和右端；`LPOP`命令和`RPOP`命令分别用于从列表的左端和右端弹出元素；`LINDEX`命令用于获取列表在给定位置上的一个元素；`LRANGE`命令用于获取列表在给定范围上的所有元素。

```redis
127.0.0.1:6379> rpush list-key item
(integer) 1
127.0.0.1:6379> rpush list-key item2
(integer) 2
127.0.0.1:6379> rpush list-key item
(integer) 3
127.0.0.1:6379> lrange list-key 0 -1
1) "item"
2) "item2"
3) "item"
127.0.0.1:6379> lindex list-key 1
"item2"
127.0.0.1:6379> lpop list-lkey
(nil)
127.0.0.1:6379> lrange list-key 0 -1
1) "item"
2) "item2"
3) "item"
```

向列表推入新元素之后，该命令回返回列表当前的长度。

REDIS的集合使用无序的方式存储元素。

```redis
127.0.0.1:6379> sadd set-key item
(integer) 1
127.0.0.1:6379> sadd set-key item2
(integer) 1
127.0.0.1:6379> sadd set-key item3
(integer) 1
127.0.0.1:6379> smembers set-key
1) "item2"
2) "item"
3) "item3"
127.0.0.1:6379> sismember set-key item4
(integer) 0
127.0.0.1:6379> sismember set-key item
(integer) 1
127.0.0.1:6379> srem set-key item2
(integer) 1
127.0.0.1:6379> srem set-key item2
(integer) 0
127.0.0.1:6379> smembers set-key
1) "item"
2) "item3"
```

在尝试讲将一个元素添加到集合的时候，命令返回1表示这个元素被成功地添加到了集合里面，而返回0则表示这个元素已经存在于集合中

```redis
127.0.0.1:6379> hset hash-key sub-key1 value1
(integer) 1
127.0.0.1:6379> hset hash-key sub-key2 value2
(integer) 1
127.0.0.1:6379> hset hash-key sub-key1 value1
(integer) 0
127.0.0.1:6379> hgetall hash-key
1) "sub-key1"
2) "value1"
3) "sub-key2"
4) "value2"
127.0.0.1:6379> hdel hash-key sub-key2
(integer) 1
127.0.0.1:6379> hdel hash-key sub-key2
(integer) 0
127.0.0.1:6379> hget hash-key sub-key1
"value1"
127.0.0.1:6379> hgetall hash-key
1) "sub-key1"
2) "value1"
```

有序集合和散列一样，都用于存储键值对：有序集合的键被称为成员（`member`），每个成员都是各不相同的；而有序集合的值被称为分值，分值必须为浮点数。有序集合是`Redis`里面唯一一个既可以根据成员访问元素，又可以根据分值的排列顺序来访问元素的结构。

```redis
127.0.0.1:6379> zadd zset-key 728 member1
(integer) 1
127.0.0.1:6379> zadd zset-key 982 member0
(integer) 1
127.0.0.1:6379> zadd zset-key 982 member0
(integer) 0
127.0.0.1:6379> zrange zset-key 0 -1 withscores
1) "member1"
2) "728"
3) "member0"
4) "982"
127.0.0.1:6379> zrangebyscore zset-key 0 800 withscores
1) "member1"
2) "728"
127.0.0.1:6379> zrem zset-key member1
(integer) 1
127.0.0.1:6379> zrem zset-key member1
(integer) 0
127.0.0.1:6379> zrange zset-key 0 -1 withscores
1) "member0"
2) "982"
```

`ZRANGEBYSCORE`获取有序集合在给定分值范围内的所有元素

网站计算评分的方法
```ruby
def article_vote(client, user, article)
  cutoff = Time.now.to_i - ONE_WEEK_IN_SECONDS
  return if client.zscore('time:', article) < cutoff

  article_id = article.split(':')[-1]

  if client.sadd("voted:#{article_id}", user)
    client.zincrby('score:', VOTE_SCORE, article)
    client.hincrby(article, 'votes', 1)
  end
end
```